In [190]:
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
load_dotenv()

True

In [191]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

In [292]:
proyec = {"name":1,"category_code": 1, "_id": 0,"total_money_raised":1,  "offices.city":1, "offices.latitude":1, "offices.longitude":1}
cond = {"category_code": "games_video", "total_money_raised":{"$ne":"$0"}, "offices.latitude":{"$type": "double"},"offices.longitude":{"$type": "double"}}
list(companies.find(cond,proyec))[8]

{'name': 'Ustream',
 'category_code': 'games_video',
 'total_money_raised': '$60.1M',
 'offices': [{'city': 'San Francisco',
   'latitude': 37.392936,
   'longitude': -122.07948},
  {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
  {'city': 'Budapest', 'latitude': None, 'longitude': None}]}

In [293]:
selec = pd.DataFrame(list(companies.find(cond,proyec)))

In [294]:
#lista[1]["offices"][0]["city"] #len(lista[1]["offices"])
#lista[1]["offices"][0]["latitude"] 
#lista[1]["offices"][0]["longitude"] 

In [295]:
selec.total_money_raised = selec.total_money_raised.str.strip()

In [296]:
selec["amount"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[1]
selec["mil"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[2]
selec["currency"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[0]

In [297]:
selec.currency.unique()

array(['$', '€', '£'], dtype=object)

In [337]:
selec.mil.unique()

array(['M', 'k'], dtype=object)

In [298]:
selec.head()

,name,category_code,total_money_raised,offices,amount,mil,currency
0,Lala,games_video,$44.2M,"[{'city': 'Palo Alto', 'latitude': 37.451151, ...",44.2,M,$
1,Joost,games_video,$45M,"[{'city': 'New York', 'latitude': 40.7464969, ...",45,M,$
2,Babelgum,games_video,$13.2M,"[{'city': 'London', 'latitude': 53.344104, 'lo...",13.2,M,$
3,Kyte,games_video,$23.4M,"[{'city': 'San Francisco', 'latitude': 37.7884...",23.4,M,$
4,Veoh,games_video,$69.8M,"[{'city': 'San Diego', 'latitude': 32.902266, ...",69.8,M,$


In [310]:
dol = selec[(selec["currency"] == '$') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
dol

,name,category_code,total_money_raised,offices,amount,mil,currency
85,FameCast,games_video,$9.7M,"[{'city': 'Austin', 'latitude': 30.268735, 'lo...",9.7,M,$


In [305]:
eur = selec[(selec["currency"] == '€') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
eur

,name,category_code,total_money_raised,offices,amount,mil,currency
242,Bigpoint,games_video,€420M,"[{'city': 'Hamburg', 'latitude': 53.5973055, '...",420,M,€


In [309]:
pounds = selec[(selec["currency"] == '£') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
pounds

,name,category_code,total_money_raised,offices,amount,mil,currency
125,amazingtunes,games_video,£1.4M,"[{'city': 'Newcastle Upon Tyne', 'latitude': 5...",1.4,M,£


In [351]:
aus = list(dol.offices.values)[0]
aus_lat = aus[0]["latitude"]
aus_lon = austin[0]["longitude"]
aus_lat, aus_lon

(30.268735, -97.745209)

In [352]:
ham = list(eur.offices.values)[0]
ham_lat = ham[0]["latitude"]
ham_lon = ham[0]["longitude"]
ham_lat, ham_lon

(53.5973055, 9.9762519)

In [353]:
ncas = list(pounds.offices.values)[0]
ncas_lat = ncas[0]["latitude"]
ncas_lon = ncas[0]["longitude"]
ncas_lat,ncas_lon

(54.971346, -1.611141)

## localizar puntos en fs

In [354]:
client_id = os.getenv("four")
client_secret = os.getenv("foursec")

In [355]:
#madrid_porsifalla = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}
Austin = {'type': 'Point', 'coordinates': [aus_lat, aus_lon]}
Hamburg = {'type': 'Point', 'coordinates': [ham_lat, ham_lon]}
Newcastle = {'type': 'Point', 'coordinates': [ncas_lat, ncas_lon]}

In [357]:
Austin, Hamburg, Newcastle

({'type': 'Point', 'coordinates': [30.268735, -97.745209]},
 {'type': 'Point', 'coordinates': [53.5973055, 9.9762519]},
 {'type': 'Point', 'coordinates': [54.971346, -1.611141]})

In [358]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

In [368]:
def parametrizacion(place,store):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    parametros = {
        "client_id": client_id,
        "client_secret": client_secret,
        "v": "20180323",
        "ll": f"{place['coordinates'][0]}, {place['coordinates'][1]}",
        "query": store,
        "radius": 2500
    }
    return requests.get(url_query, params = parametros).json()

In [373]:
place = Hamburg
store = "Basketball"

Ham_bas = parametrizacion(place,store)["response"]["venues"]

In [378]:
Ham_bas[0]["location"]["distance"]

1218

In [381]:
place = Hamburg
store = "Conference"
ham_talks = parametrizacion(place,store)["response"]["venues"]

In [408]:
place = Hamburg
store = "Nursery"
ham_school = parametrizacion(place,store)["response"]["venues"]

In [409]:
ham_school

[]